In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [39]:
import tensorflow as tf
import numpy as np
import json
from glob import glob

In [3]:
PAD = 0
EOS = 1
UNK = 2
GO = 3
[PAD, EOS, UNK, GO]

[0, 1, 2, 3]

In [4]:
left_dict = {0: 0,
 1: 1,
 2: 2,
 3: 3,
 ' ': 4,
 '!': 5,
 '"': 6,
 "'": 7,
 '(': 8,
 ')': 9,
 '+': 10,
 ',': 11,
 '-': 12,
 '.': 13,
 '0': 14,
 '1': 15,
 '2': 16,
 '3': 17,
 '4': 18,
 '5': 19,
 '6': 20,
 '7': 21,
 '8': 22,
 '9': 23,
 ':': 24,
 ';': 25,
 '?': 26,
 'A': 27,
 'B': 28,
 'C': 29,
 'D': 30,
 'E': 31,
 'F': 32,
 'G': 33,
 'H': 34,
 'I': 35,
 'J': 36,
 'K': 37,
 'L': 38,
 'M': 39,
 'N': 40,
 'O': 41,
 'P': 42,
 'Q': 43,
 'R': 44,
 'S': 45,
 'T': 46,
 'U': 47,
 'V': 48,
 'W': 49,
 'X': 50,
 'Y': 51,
 'Z': 52,
 'a': 53,
 'b': 54,
 'c': 55,
 'd': 56,
 'e': 57,
 'f': 58,
 'g': 59,
 'h': 60,
 'i': 61,
 'j': 62,
 'k': 63,
 'l': 64,
 'm': 65,
 'n': 66,
 'o': 67,
 'p': 68,
 'q': 69,
 'r': 70,
 's': 71,
 't': 72,
 'u': 73,
 'v': 74,
 'w': 75,
 'x': 76,
 'y': 77,
 'z': 78}

In [5]:
right_dict = {0: 0,
 1: 1,
 2: 2,
 3: 3,
 ' ': 4,
 '!': 5,
 '"': 6,
 '-': 7,
 '.': 8,
 ':': 9,
 ';': 10,
 '،': 11,
 '؟': 12,
 'ء': 13,
 'آ': 14,
 'أ': 15,
 'ؤ': 16,
 'إ': 17,
 'ئ': 18,
 'ا': 19,
 'ب': 20,
 'ة': 21,
 'ت': 22,
 'ث': 23,
 'ج': 24,
 'ح': 25,
 'خ': 26,
 'د': 27,
 'ذ': 28,
 'ر': 29,
 'ز': 30,
 'س': 31,
 'ش': 32,
 'ص': 33,
 'ض': 34,
 'ط': 35,
 'ظ': 36,
 'ع': 37,
 'غ': 38,
 'ف': 39,
 'ق': 40,
 'ك': 41,
 'ل': 42,
 'م': 43,
 'ن': 44,
 'ه': 45,
 'و': 46,
 'ى': 47,
 'ي': 48,
 'ّ': 49,
 'ٓ': 50,
 '٠': 51,
 '١': 52,
 '٢': 53,
 '٣': 54,
 '٤': 55,
 '٥': 56,
 '٦': 57,
 '٧': 58,
 '٨': 59,
 '٩': 60,
 'چ': 61,
 'ڠ': 62,
 'ڤ': 63,
 'ڬ': 64,
 'ڽ': 65,
 'ۏ': 66,
 '﴾': 67,
 '﴿': 68}
rev_right_dict = {v: k for k, v in right_dict.items()}

In [6]:
with open('jawi-set.json') as fopen:
    jawi = json.load(fopen)
jawi['train'][:10]

['كاوسن كڤولاوان سڤراتلي يڠ',
 'ڤرليمين ڤرسكوتوان اونتوق',
 'ڤنوبوهن تامن سينر',
 'ڤريڠكت كمنترين، كاتڽ.',
 'تله مندرم سباڽق تيڬ',
 'هاري اين،',
 'برتوليرنسي"',
 'مڠيسهكان',
 'سوڠ-قواڠ',
 'سيبير دڠن باجو بياسا.']

In [7]:
with open('rumi-set.json') as fopen:
    rumi = json.load(fopen)

In [16]:
train_X = rumi['train']
train_Y = jawi['train']

test_X = rumi['test']
test_Y = jawi['test']

In [9]:
class Translator:
    def __init__(self, size_layer, num_layers, embedded_size,
                 from_dict_size, to_dict_size, learning_rate, beam_width = 10):
        
        def cells(reuse=False):
            return tf.nn.rnn_cell.LSTMCell(size_layer,initializer=tf.orthogonal_initializer(),reuse=reuse)
        
        def attention(encoder_out, seq_len, reuse=False):
            attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(num_units = size_layer, 
                                                                    memory = encoder_out,
                                                                    memory_sequence_length = seq_len)
            return tf.contrib.seq2seq.AttentionWrapper(
            cell = tf.nn.rnn_cell.MultiRNNCell([cells(reuse) for _ in range(num_layers)]), 
                attention_mechanism = attention_mechanism,
                attention_layer_size = size_layer)
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype=tf.int32)
        self.Y_seq_len = tf.count_nonzero(self.Y, 1, dtype=tf.int32)
        batch_size = tf.shape(self.X)[0]
        
        encoder_embedding = tf.Variable(tf.random_uniform([from_dict_size, embedded_size], -1, 1))
        decoder_embedding = tf.Variable(tf.random_uniform([to_dict_size, embedded_size], -1, 1))
        
        encoder_out, encoder_state = tf.nn.dynamic_rnn(
            cell = tf.nn.rnn_cell.MultiRNNCell([cells() for _ in range(num_layers)]), 
            inputs = tf.nn.embedding_lookup(encoder_embedding, self.X),
            sequence_length = self.X_seq_len,
            dtype = tf.float32)
        main = tf.strided_slice(self.Y, [0, 0], [batch_size, -1], [1, 1])
        decoder_input = tf.concat([tf.fill([batch_size, 1], GO), main], 1)
        dense = tf.layers.Dense(to_dict_size)
        
        with tf.variable_scope('decode'):
            decoder_cells = attention(encoder_out, self.X_seq_len)
            training_helper = tf.contrib.seq2seq.TrainingHelper(
                inputs = tf.nn.embedding_lookup(decoder_embedding, decoder_input),
                sequence_length = self.Y_seq_len,
                time_major = False)
            training_decoder = tf.contrib.seq2seq.BasicDecoder(
                    cell = decoder_cells,
                    helper = training_helper,
                    initial_state = decoder_cells.zero_state(batch_size, tf.float32).clone(cell_state=encoder_state),
                    output_layer = dense)
            training_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                    decoder = training_decoder,
                    impute_finished = True,
                    maximum_iterations = tf.reduce_max(self.Y_seq_len))
            self.training_logits = training_decoder_output.rnn_output
            
        with tf.variable_scope('decode', reuse=True):
            predicting_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
                embedding = decoder_embedding,
                start_tokens = tf.tile(tf.constant([GO], dtype=tf.int32), [batch_size]),
                end_token = EOS)
            predicting_decoder = tf.contrib.seq2seq.BasicDecoder(
                    cell = decoder_cells,
                    helper = predicting_helper,
                    initial_state = decoder_cells.zero_state(batch_size, tf.float32).clone(cell_state=encoder_state),
                    output_layer = dense)
            predicting_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                    decoder = predicting_decoder,
                    impute_finished = True,
                    maximum_iterations = 2 * tf.reduce_max(self.X_seq_len))
            self.greedy = predicting_decoder_output.sample_id
            self.greedy = tf.identity(self.greedy,name='greedy')
        
        with tf.variable_scope('decode', reuse=True):
            
            encoder_out_tiled = tf.contrib.seq2seq.tile_batch(encoder_out, beam_width)
            encoder_state_tiled = tf.contrib.seq2seq.tile_batch(encoder_state, beam_width)
            X_seq_len_tiled = tf.contrib.seq2seq.tile_batch(self.X_seq_len, beam_width)
            decoder_cell = attention(encoder_out_tiled, X_seq_len_tiled, reuse=True)
            
            predicting_decoder = tf.contrib.seq2seq.BeamSearchDecoder(
                cell = decoder_cell,
                embedding = decoder_embedding,
                start_tokens = tf.tile(tf.constant([GO], dtype=tf.int32), [batch_size]),
                end_token = EOS,
                initial_state = decoder_cell.zero_state(batch_size * beam_width, tf.float32).clone(
                    cell_state = encoder_state_tiled),
                beam_width = beam_width,
                output_layer = dense,
                length_penalty_weight = 0.0)
            
            predicting_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                decoder = predicting_decoder,
                impute_finished = False,
                maximum_iterations = tf.reduce_max(self.X_seq_len))
            
            self.beam = predicting_decoder_output.predicted_ids[:, :, 0]
            self.beam = tf.identity(self.beam,name='beam')
        
        masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)
        self.masks = masks
        self.cost = tf.contrib.seq2seq.sequence_loss(logits = self.training_logits,
                                                     targets = self.Y,
                                                     weights = masks)
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        y_t = tf.argmax(self.training_logits,axis=2)
        y_t = tf.cast(y_t, tf.int32)
        self.prediction = tf.boolean_mask(y_t, masks)
        mask_label = tf.boolean_mask(self.Y, masks)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [10]:
size_layer = 256
num_layers = 2
embedded_size = 256
learning_rate = 1e-3
batch_size = 32
epoch = 20

In [11]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Translator(size_layer, num_layers, embedded_size, len(left_dict), len(right_dict), learning_rate)
sess.run(tf.global_variables_initializer())







The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



2022-06-01 10:54:13.304553: I tensorflow/core/platform/profile_utils/cpu_utils.cc:109] CPU Frequency: 2496000000 Hz
2022-06-01 10:54:13.304949: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x3b5d1c0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-06-01 10:54:13.304962: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-06-01 10:54:13.306018: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-06-01 10:54:13.308229: E tensorflow/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-06-01 10:54:13.308244: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: huseincomel-desktop
2022-06-01 10:54:13.308248: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: huseincomel

In [12]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'lstm-rumi-jawi/model.ckpt')

'lstm-rumi-jawi/model.ckpt'

In [13]:
def pad_sentence_batch(sentence_batch, pad_int):
    padded_seqs = []
    seq_lens = []
    max_sentence_len = max([len(sentence) for sentence in sentence_batch])
    for sentence in sentence_batch:
        padded_seqs.append(sentence + [pad_int] * (max_sentence_len - len(sentence)))
        seq_lens.append(len(sentence))
    return padded_seqs, seq_lens

In [29]:
batch_x = [[left_dict[c] for c in s] + [1] for s in train_X[i: i + 5]]
batch_y = [[right_dict[c] for c in s] + [1] for s in train_Y[i: i + 5]]
batch_x, _ = pad_sentence_batch(batch_x, 0)
batch_y, _ = pad_sentence_batch(batch_y, 0)

In [30]:
np.array(batch_y).shape

(5, 20)

In [31]:
np.array(batch_x).shape

(5, 29)

In [32]:
%%time

sess.run([model.cost, model.optimizer], feed_dict = {model.X: batch_x, model.Y: batch_y})

CPU times: user 496 ms, sys: 7.86 ms, total: 504 ms
Wall time: 55.8 ms


[4.0693455, None]

In [33]:
greedy, beam = sess.run([model.greedy, model.beam], feed_dict = {model.X: batch_x[:1]})

In [34]:
greedy, beam

(array([[44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44,
         44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44,
         44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44,
         44, 44, 44, 44, 44, 44, 44, 44, 44, 44]], dtype=int32),
 array([[46, 44, 44, 44, 44, 44, 44,  4, 44, 44, 44, 44, 44, 44,  4, 44,
         44, 44, 44, 44,  4, 44, 44, 44, 44,  4, 44, 44, 44]], dtype=int32))

In [40]:
import tqdm
from sklearn.utils import shuffle

for e in range(2):
    train_X, train_Y = shuffle(train_X, train_Y)
    pbar = tqdm.tqdm(
        range(0, len(train_X), batch_size), desc = 'minibatch loop')
    train_loss, train_acc, test_loss, test_acc = [], [], [], []
    for i in pbar:
        index = min(i + batch_size, len(train_X))
        batch_x = [[left_dict[c] for c in s] + [1] for s in train_X[i: index]]
        batch_y = [[right_dict[c] for c in s] + [1] for s in train_Y[i: index]]
        batch_x, _ = pad_sentence_batch(batch_x, PAD)
        batch_y, _ = pad_sentence_batch(batch_y, PAD)
        feed = {model.X: batch_x,
                model.Y: batch_y}
        accuracy, loss, _ = sess.run([model.accuracy,model.cost,model.optimizer],
                                    feed_dict = feed)
        train_loss.append(loss)
        train_acc.append(accuracy)
        pbar.set_postfix(cost = loss, accuracy = accuracy)
    
    pbar = tqdm.tqdm(
        range(0, len(test_X), batch_size), desc = 'minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_X))
        batch_x = [[left_dict[c] for c in s] + [1] for s in test_X[i: index]]
        batch_y = [[right_dict[c] for c in s] + [1] for s in test_Y[i: index]]
        batch_x, _ = pad_sentence_batch(batch_x, PAD)
        batch_y, _ = pad_sentence_batch(batch_y, PAD)
        feed = {model.X: batch_x,
                model.Y: batch_y,}
        accuracy, loss = sess.run([model.accuracy,model.cost],
                                    feed_dict = feed)

        test_loss.append(loss)
        test_acc.append(accuracy)
        pbar.set_postfix(cost = loss, accuracy = accuracy)
    
    print('epoch %d, training avg loss %f, training avg acc %f'%(e+1,
                                                                 np.mean(train_loss),np.mean(train_acc)))
    print('epoch %d, testing avg loss %f, testing avg acc %f'%(e+1,
                                                              np.mean(test_loss),np.mean(test_acc)))

minibatch loop:  31%|███       | 4768/15566 [17:19<35:09,  5.12it/s, accuracy=1, cost=0.00731]     IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

minibatch loop:  65%|██████▍   | 10087/15566 [36:43<20:23,  4.48it/s, accuracy=0.998, cost=0.00692]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

minibatch loop:  94%|█████████▍| 14691/15566 [53:33<03:08,  4.63it/s, accuracy=0.993, cost=0.0192] IOPub message rate exceeded.
The notebook server will

epoch 2, training avg loss 0.027398, training avg acc 0.992040
epoch 2, testing avg loss 0.026345, testing avg acc 0.992578


In [41]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'lstm-bahdanau-rumi-jawi/model.ckpt')

'lstm-bahdanau-rumi-jawi/model.ckpt'

In [42]:
string = 'comel'
batch = [left_dict[c] for c in string] + [1]

In [43]:
greedy, beam = sess.run([model.greedy, model.beam], feed_dict = {model.X: [batch]})

In [44]:
greedy, beam

(array([[61, 46, 43, 48, 42,  1]], dtype=int32),
 array([[61, 46, 43, 48, 42,  1]], dtype=int32))

In [45]:
''.join([rev_right_dict[i] for i in greedy[0] if i > 3])

'چوميل'

In [46]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'Placeholder' in n.name
        or 'greedy' in n.name
        or 'beam' in n.name
        or 'alphas' in n.name)
        and 'Adam' not in n.name
        and 'beta' not in n.name
        and 'OptimizeLoss' not in n.name
        and 'Global_Step' not in n.name
    ]
)
strings.split(',')

['Placeholder',
 'Placeholder_1',
 'Variable',
 'Variable_1',
 'rnn/multi_rnn_cell/cell_0/lstm_cell/kernel',
 'rnn/multi_rnn_cell/cell_0/lstm_cell/bias',
 'rnn/multi_rnn_cell/cell_1/lstm_cell/kernel',
 'rnn/multi_rnn_cell/cell_1/lstm_cell/bias',
 'decode/memory_layer/kernel',
 'decode/decoder/attention_wrapper/multi_rnn_cell/cell_0/lstm_cell/kernel',
 'decode/decoder/attention_wrapper/multi_rnn_cell/cell_0/lstm_cell/bias',
 'decode/decoder/attention_wrapper/multi_rnn_cell/cell_1/lstm_cell/kernel',
 'decode/decoder/attention_wrapper/multi_rnn_cell/cell_1/lstm_cell/bias',
 'decode/decoder/attention_wrapper/bahdanau_attention/query_layer/kernel',
 'decode/decoder/attention_wrapper/bahdanau_attention/attention_v',
 'decode/decoder/attention_wrapper/attention_layer/kernel',
 'decode/decoder/dense/kernel',
 'decode/decoder/dense/bias',
 'decode_1/greedy',
 'decode_2/decoder/while/BeamSearchDecoderStep/beam_width',
 'decode_2/decoder/while/BeamSearchDecoderStep/next_beam_probs/range/start',
 

In [47]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            "directory: %s" % model_dir)

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path
    
    absolute_model_dir = "/".join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + "/frozen_model.pb"
    clear_devices = True
    with tf.Session(graph=tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(input_checkpoint + '.meta', clear_devices=clear_devices)
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(",")
        ) 
        with tf.gfile.GFile(output_graph, "wb") as f:
            f.write(output_graph_def.SerializeToString())
        print("%d ops in the final graph." % len(output_graph_def.node))

In [48]:
freeze_graph("lstm-bahdanau-rumi-jawi", strings)




INFO:tensorflow:Restoring parameters from lstm-bahdanau-rumi-jawi/model.ckpt
INFO:tensorflow:Froze 16 variables.
INFO:tensorflow:Converted 16 variables to const ops.

1649 ops in the final graph.


In [49]:
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

In [50]:
g=load_graph('lstm-bahdanau-rumi-jawi/frozen_model.pb')

In [51]:
x = g.get_tensor_by_name('import/Placeholder:0')
i_greedy = g.get_tensor_by_name('import/decode_1/greedy:0')
i_beam = g.get_tensor_by_name('import/decode_2/beam:0')

In [52]:
test_sess = tf.InteractiveSession(graph=g)

/home/ubuntu/tf-nvidia/lib/python3.8/site-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [53]:
greedy, beam = test_sess.run([i_greedy, i_beam], feed_dict = {x: [batch]})

In [54]:
''.join([rev_right_dict[i] for i in greedy[0] if i > 3])

'چوميل'

In [55]:
from tensorflow.tools.graph_transforms import TransformGraph
from tensorflow.contrib.seq2seq.python.ops import beam_search_ops

In [57]:
transforms = ['add_default_attributes',
             'remove_nodes(op=Identity, op=CheckNumerics, op=Dropout)',
             'fold_batch_norms',
             'fold_old_batch_norms',
             'quantize_weights(fallback_min=-10, fallback_max=10)',
             'strip_unused_nodes',
             'sort_by_execution_order']

pb = 'lstm-bahdanau-rumi-jawi/frozen_model.pb'
input_graph_def = tf.GraphDef()
with tf.gfile.FastGFile(pb, 'rb') as f:
    input_graph_def.ParseFromString(f.read())

transformed_graph_def = TransformGraph(input_graph_def, 
                                       ['Placeholder'],
                                       ['decode_1/greedy', 'decode_2/beam'], transforms)

with tf.gfile.GFile(f'{pb}.quantized', 'wb') as f:
    f.write(transformed_graph_def.SerializeToString())

2022-06-01 15:34:36.865453: I tensorflow/tools/graph_transforms/transform_graph.cc:318] Applying add_default_attributes
2022-06-01 15:34:36.870972: I tensorflow/tools/graph_transforms/transform_graph.cc:318] Applying remove_nodes
2022-06-01 15:34:36.876625: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for decode_1/greedy
2022-06-01 15:34:36.877407: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for decode_2/beam
2022-06-01 15:34:36.884702: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for decode_1/greedy
2022-06-01 15:34:36.885274: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for decode_2/beam
2022-06-01 15:34:36.891508: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for decode_1/greedy
2022-06-01 15:34:36.892066: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for decode_2/beam
2022-06-01 15:34:36.911687

In [58]:
!tar -cvf lstm-bahdanau-rumi-jawi.tar lstm-bahdanau-rumi-jawi

lstm-bahdanau-rumi-jawi/
lstm-bahdanau-rumi-jawi/checkpoint
lstm-bahdanau-rumi-jawi/frozen_model.pb.quantized
lstm-bahdanau-rumi-jawi/model.ckpt.index
lstm-bahdanau-rumi-jawi/model.ckpt.data-00000-of-00001
lstm-bahdanau-rumi-jawi/model.ckpt.meta
lstm-bahdanau-rumi-jawi/frozen_model.pb


In [59]:
from malaya_boilerplate.huggingface import upload_dict

/home/ubuntu/tf-nvidia/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [60]:
files_mapping = {'lstm-bahdanau-rumi-jawi/frozen_model.pb': 'model.pb'}
upload_dict(model = 'rumi-jawi-lstm-bahdanau', files_mapping = files_mapping)

/home/ubuntu/tf-nvidia/lib/python3.8/site-packages/huggingface_hub/hf_api.py:79: FutureWarning: `name` and `organization` input arguments are deprecated and will be removed in v0.8. Pass `repo_id` instead.
  warnings.warn(


In [61]:
files_mapping = {'lstm-bahdanau-rumi-jawi/frozen_model.pb.quantized': 'model.pb'}
upload_dict(model = 'rumi-jawi-lstm-bahdanau-quantized', files_mapping = files_mapping)

In [62]:
files_mapping = {'lstm-bahdanau-rumi-jawi.tar': 'lstm-bahdanau-rumi-jawi.tar'}
upload_dict(model = 'pretrained-rumi-jawi', files_mapping = files_mapping)

In [85]:
# !~/tf-nvidia/bin/pip3 install python-Levenshtein

def calculate_cer(actual, hyp):
    """
    Calculate CER using `python-Levenshtein`.
    """
    import Levenshtein as Lev

    actual = actual.replace(' ', '')
    hyp = hyp.replace(' ', '')
    return Lev.distance(actual, hyp) / len(actual)


def calculate_wer(actual, hyp):
    """
    Calculate WER using `python-Levenshtein`.
    """
    import Levenshtein as Lev

    b = set(actual.split() + hyp.split())
    word2char = dict(zip(b, range(len(b))))

    w1 = [chr(word2char[w]) for w in actual.split()]
    w2 = [chr(word2char[w]) for w in hyp.split()]

    return Lev.distance(''.join(w1), ''.join(w2)) / len(actual.split())

def decode(ids):
    return ''.join([rev_right_dict[i] for i in ids if i > 3])

In [66]:
i = 0
index = 10
batch_x = [[left_dict[c] for c in s] + [1] for s in test_X[i: index]]
batch_y = [[right_dict[c] for c in s] + [1] for s in test_Y[i: index]]
batch_x, _ = pad_sentence_batch(batch_x, PAD)
batch_y, _ = pad_sentence_batch(batch_y, PAD)
feed = {model.X: batch_x,
        model.Y: batch_y,}
greedy, beam = sess.run([model.greedy, model.beam], feed_dict = feed)

In [88]:
wer, cer = [], []
for k in range(len(batch_x)):
    d_left = decode(batch_y[k])
    d_right = decode(greedy[k])
    wer.append(calculate_wer(d_left, d_right))
    cer.append(calculate_cer(d_left, d_right))
    
np.mean(wer), np.mean(cer)

(0.03333333333333333, 0.006666666666666666)

In [92]:
wer, cer = [], []
for i in tqdm.tqdm(range(0, len(test_X), batch_size)):
    index = min(i + batch_size, len(test_X))
    batch_x = [[left_dict[c] for c in s] + [1] for s in test_X[i: index]]
    batch_y = [[right_dict[c] for c in s] + [1] for s in test_Y[i: index]]
    batch_x, _ = pad_sentence_batch(batch_x, PAD)
    batch_y, _ = pad_sentence_batch(batch_y, PAD)
    feed = {model.X: batch_x,
            model.Y: batch_y,}
    greedy, beam = sess.run([model.greedy, model.beam], feed_dict = feed)
    for k in range(len(batch_x)):
        d_left = decode(batch_y[k])
        d_right = decode(greedy[k])
        wer.append(calculate_wer(d_left, d_right))
        cer.append(calculate_cer(d_left, d_right))

100%|██████████| 1730/1730 [09:41<00:00,  2.98it/s]


In [93]:
np.mean(wer), np.mean(cer)

(0.06737832963079593, 0.014847105998349451)